In [1]:
from pyspecter.SPECTER import SPECTER
import numpy as np


In [2]:
specter = SPECTER()

No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


Compiling SPECTER model...
Generating test events for tracing ...
Test events generated! Time taken:  4.147363901138306  seconds.
Compiling spectral representation functions ...
Compilation complete! Time taken:  42.77806878089905  seconds.


# Test 1: Checking that the SEMD between an event and itself is zero!

In [3]:
# Load csv file
event = np.loadtxt("test_event.csv", delimiter=",", skiprows=1)

# Reshape to (1, 94, 3)
event = event.reshape(1, -1, 3)

emd_euc = specter.spectralEMD(event,  event)
emd_cylinder = specter.spectralEMD(event, event, metric="cylindrical")
print("Euclidean EMD: ", emd_euc)
print("Cylinder EMD: ", emd_cylinder)

Euclidean EMD:  [-4.7683716e-07]
Cylinder EMD:  [-4.7683716e-07]


# Test 2: Euclidean versus Cylindrical SEMD

In [12]:
# These particles have a distance of 2pi. On the cylinder, they are at the same position.
event1 = np.array([[0.5, 0, 0], [0.5, 0, 2 * np.pi]])
event2 = np.array([[1.0, 0, 0]])

# Add a batch dimension
event1 = event1.reshape(1, -1, 3)
event2 = event2.reshape(1, -1, 3)


emd_euc = specter.spectralEMD(event1, event2)
emd_cylinder = specter.spectralEMD(event1, event2, metric="cylindrical")
print("Euclidean EMD: ", emd_euc, ", Expected answer = (2pi)^2/2 = " + str((2 * np.pi) ** 2 / 2))
print("Cylinder EMD: ", emd_cylinder)

Euclidean EMD:  [19.73921] , Expected answer = (2pi)^2/2 = 19.739208802178716
Cylinder EMD:  [0.]


# Test 3: SEMD with non-normalized energies

The SEMD should scale with E_tot^2. We can test this by taking events with E_tot = 1, and then multipling by some constant

In [17]:
event1 = np.array([[0.5, 0, 0], [0.5, 0, 1]])
event2 = np.array([[1.0, 0, 0]])

# Add a batch dimension
event1 = event1.reshape(1, -1, 3)
event2 = event2.reshape(1, -1, 3)


emd1 = specter.spectralEMD(event1, event2)
multiplier = 3

modified_event1 = event1.copy()
modified_event1[:,:, 0] = modified_event1[:, :, 0] * multiplier
modified_event2 = event2.copy()
modified_event2[:,:, 0] = modified_event2[:, :, 0] * multiplier
emd2 = specter.spectralEMD(modified_event1, modified_event2)

print(f"Ratio of EMDs: ", emd2 / emd1, f", Expected answer = {multiplier}^2 = {multiplier**2}")

Ratio of EMDs:  [9.] , Expected answer = 3^2 = 9


# Test 4: Unabalanced events


In [10]:
omega_max = 1

# Event 1 is a single particle with energy 1
event1 = np.array([[2.0, 0, 0]])

# Event 2 is a single particle with energy 2
event2 = np.array([[1.0, 0, 0]])

# Add a batch dimension
event1 = event1.reshape(1, -1, 3)
event2 = event2.reshape(1, -1, 3)

# Calculate the EMD between the two events. THIS WILL FAIL, SINCE NO OMEGA IS PROVIDED. THIS IS EXPECTED!
try:
    emd = specter.spectralEMD(event1, event2)
except:
    print("Calculation failed since the energies are not equal. Please provide omega_max.")


# Calculate the EMD between the two events. THIS WILL WORK, SINCE OMEGA IS PROVIDED!
emd = specter.spectralEMD(event1, event2, omega_max=omega_max)
print("EMD: ", emd, ", Expected answer = (omega^2)|(e1^2 - e2^2)| = ", (omega_max)**2 * (event1[0,0,0]**2 - event2[0,0,0]**2))

Calculation failed since the energies are not equal. Please provide omega_max.
EMD:  [3.] , Expected answer = (omega^2)|(e1^2 - e2^2)| =  3.0
